In [1]:
import torch 
from torch import nn 
from einops import rearrange
import math

import matplotlib.pyplot as plt

# load filters of pretrained model
n_layers = 18
d_model = 108
l_max = 2048
multihyena_pretrained_filters = torch.load('multihyena_pretrained_filters.pt')

In [2]:

fig, axs = plt.subplots(3, 6, figsize=(20, 10))
skip = 10
norm = torch.norm(multihyena_pretrained_filters, dim=[-1, -2], p=float("inf"))

for k in range(n_layers):
    ax = axs[k // 6, k % 6]
    ax.plot(torch.arange(l_max)[::skip], multihyena_pretrained_filters[k, :, ::skip].T / norm[k], 'k', alpha=0.7)
    ax.set_title(f"Layer {k}")
    ax.set_xlabel("d")
    ax.set_xlim([0, l_max])
fig.tight_layout()

In [3]:

class LaughingHyenaFilter(nn.Module):
    def __init__(
        self,
        order: int,
        num_filters: int,
        r_min: float = 0.9,
        r_max: float = 0.99,
        theta_max: float = 2 * torch.pi,
        real_fft: bool = False,
    ):
        super().__init__()
        self.order = order
        self.num_filters = num_filters
        self.fft = torch.fft.rfft if real_fft else torch.fft.fft
        self.register_parameter("R_re", nn.Parameter(torch.ones(order, num_filters)))
        self.register_parameter("R_im", nn.Parameter(torch.ones(order, num_filters)))
        self.register_parameter("r", nn.Parameter(torch.ones(order, num_filters)))
        self.register_parameter("theta", nn.Parameter(torch.ones(order, num_filters)))
        self.register_buffer("h_0", torch.zeros(1, num_filters))

        self._init_params(r_max, r_min, theta_max)

    def _init_params(self, r_max, r_min, theta_max):
        # Init poles distributed uniformly of ring of the complex plane
        # between r_min and r_max and phase between 0 and theta_max
        u1 = torch.rand(self.order, self.num_filters)
        u2 = torch.rand(self.order, self.num_filters)
        self.r.data = r_min + (r_max - r_min) * u1
        self.theta.data = theta_max * u2
        # Init residues with Glorot initialization
        self.R_re.data = torch.randn(self.order, self.num_filters) * math.sqrt(
            1 / self.order
        )
        self.R_im.data = torch.randn(self.order, self.num_filters) * math.sqrt(
            1 / self.order
        )

    def _get_poles_and_residues(self):
        # poles
        p = self.r * torch.exp(1j * self.theta)
        # residues
        R = self.R_re + 1j * self.R_im
        return p, R

    def forward(self, L):
        p, R = self._get_poles_and_residues()
        t = torch.arange(L-1).unsqueeze(1).unsqueeze(2).to(p)
        h = torch.sum(R * p**t, dim=1).real
        h = rearrange(h, "l n -> n l")
        return h

    def filter(self, L, *args, **kwargs):
        # evaluate filter for t = 1, ..., L
        h = self(L)
        # stack h_0 to the beginning of the filter
        h = torch.cat([self.h_0.to(h), h], dim=1)
        h = rearrange(h, "n l -> 1 n l")
        return h

In [4]:
# we use l2 loss for distillation
loss_fn = lambda x, y: torch.mean((x - y).abs().pow_(2))

from tqdm import trange

def fit(filter : torch.Tensor,
        distilled_filter : nn.Module,
        train_cfg : dict
        ) -> list:
    """
        Args:
            filter: a filter to be distilled [num filters, seq len]
            laughing_hyena_filter: a laughing hyena filter to be trained [nn.Module]
            train_cfg: a dictionary with training parameters
    """
    device = train_cfg["device"]
    filter = filter.detach().to(device)
    with torch.no_grad():
        # store filter[:1] and keep only filter[1:]
        h_0 = filter[:,:1]
        target = filter[:,1:]    
    # assign h_0 to distilled_mixer
    distilled_filter.h_0 = h_0

    opt = torch.optim.AdamW(
        distilled_filter.parameters(), lr=6e-4, weight_decay=0.0, betas=(0.9, 0.99)
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        opt, T_0=train_cfg["epochs"], T_mult=1, eta_min=1e-6
    )
    
    log = []
    if train_cfg["verbose"] == True:
        for i in trange(train_cfg["epochs"]):
            opt.zero_grad()
            pred = distilled_filter(l_max)
            loss = loss_fn(target, pred)
            loss.backward()
            log.append(loss.item())
            opt.step()
            scheduler.step()
            if train_cfg["verbose"] == True and i % train_cfg["log_interval"] == 0:
                print(f"Epoch {i}: loss = {loss.item()}")
    return log

def distill_hyena(pretrained_filters, distill_cfg, train_cfg, save=True):

    distilled_filters = [
        LaughingHyenaFilter(**distill_cfg).to(train_cfg['device']) for _ in range(pretrained_filters.shape[0])
    ]

    train_logs = []
    for k, (pretrained_filter, distilled_filter) in enumerate(
        zip(pretrained_filters, distilled_filters)
    ):
        print(f"Starting distillation of mixer {k} with order {distill_cfg['order']} state-space models...")
        log = fit(pretrained_filter, distilled_filter, train_cfg)
        train_logs.append(log)
    return distilled_filters, train_logs

In [5]:
distill_cfg = {
    "order": 16,
    "num_filters": 108,
}

train_cfg = {"epochs": 30000, "device": "cuda:1", "verbose": True, "log_interval": 10000}
#multihyena_pretrained_filters = rearrange(torch.load("multihyena_pretrained_filters.pt"), "l n t -> 1 (l n) t")
distilled_filters, logs = distill_hyena(multihyena_pretrained_filters, distill_cfg, train_cfg)

Starting distillation of mixer 0 with order 16 state-space models...


  0%|          | 1/30000 [00:00<1:13:45,  6.78it/s]

Epoch 0: loss = 0.019439546391367912


  1%|▏         | 418/30000 [00:01<01:05, 448.41it/s]

 34%|███▎      | 10077/30000 [00:22<00:45, 436.91it/s]

...

 34%|███▎      | 10078/30000 [00:22<00:43, 455.31it/s]

Epoch 10000: loss = 0.0009626182727515697


 67%|██████▋   | 20068/30000 [00:44<00:22, 448.73it/s]

Epoch 20000: loss = 0.0009399743867106736


100%|██████████| 30000/30000 [01:06<00:00, 448.47it/s]


In [6]:
# plot loss for each layer
fig, axs = plt.subplots(3, 6, figsize=(20, 5))
losses = logs 
for k in range(n_layers):
    axs[k//6, k%6].plot(losses[k])
    axs[k//6, k%6].set_title('layer {}'.format(k))
    axs[k//6, k%6].set_xlabel('iteration')
    axs[k//6, k%6].set_ylabel('loss')
fig.tight_layout()

In [7]:
distilled_filter = torch.zeros(n_layers, d_model, l_max)
for k in range(len(distilled_filters)):
    distilled_filter[k] = distilled_filters[k].filter(l_max).detach()[0]

norm_distill = torch.norm(distilled_filter, dim=[-1, -2], p=float("inf"))

In [8]:
fig, axs = plt.subplots(3, 6, figsize=(20, 10))
skip = 10
T_plot = 100
for k in range(n_layers):
    ax = axs[k // 6, k % 6]
    ax.plot(torch.arange(l_max)[:T_plot], distilled_filter[k, ::skip, :T_plot].T , 'k', alpha=1) #/ norm_distill[k]
    ax.plot(torch.arange(l_max)[:T_plot], multihyena_pretrained_filters[k, ::skip, :T_plot].T , 'r', alpha=.5) # / norm_post[k]
    ax.set_title(f"Layer {k}")
    ax.set_xlabel("d")
    ax.set_xlim([0, T_plot])
fig.tight_layout()

In [9]:
#SAPE = 2 * torch.abs(distilled_filter - multihyena_pretrained_filters) / (torch.abs(distilled_filter) + torch.abs(multihyena_pretrained_filters))
err = torch.abs((distilled_filter - multihyena_pretrained_filters))
fig, axs = plt.subplots(3, 6, figsize=(20, 10))
skip = 10
for k in range(n_layers):
    ax = axs[k // 6, k % 6]
    ax.plot(torch.arange(l_max), err[k].mean(0), 'k', alpha=1) #
    # plot min and max bounds filled
    ax.fill_between(torch.arange(l_max), err[k].min(0).values, err[k].max(0).values, color='b', alpha=.2)
    ax.set_yscale("log")
    ax.set_title(f"Layer {k}")
    ax.set_xlabel("d")
    ax.set_xlim([0, l_max])
fig.tight_layout()